# Managing Portfolio Personal Project Calculation

In [26]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import math

In [44]:
# 1. Define Tickers, Shares, and Dates
tickers = ['VNQ', 'HYG', 'TIP', 'GOVT', 'LQD', 'IJH', 'SPY', 'VWO', 'EFA']
# Number of shares
shares = {
    'VNQ': 21,
    'HYG': 25,
    'TIP': 18,
    'GOVT': 260,
    'LQD': 36,
    'IJH': 65,
    'SPY': 16,
    'VWO': 89,
    'EFA': 74
}
# SP 500
benchmark_ticker = '^GSPC'

# Calculate 45 business days ago as the start date
start_date = (pd.Timestamp.today() - pd.tseries.offsets.BDay(45)).strftime('%Y-%m-%d')
end_date = pd.Timestamp.today().strftime('%Y-%m-%d')

print(f"Downloading data from {start_date} to {end_date}...")

# 2. Download ETF and Benchmark Price Data
data = yf.download(tickers, start=start_date, end=end_date)['Close'].dropna()
benchmark_data = yf.download(benchmark_ticker, start=start_date, end=end_date)['Close'].dropna()

print("\nHistorical Prices (last 5 rows):")
print(data.tail())

# 3. Compute Daily Returns
daily_returns = data.pct_change().dropna()
benchmark_returns = benchmark_data.pct_change().dropna()

print("\nDaily Returns (last 5 rows):")
print(daily_returns.tail())

# 4. Calculate Weighted Daily Portfolio Return

# Step 1: compute dollar value per ticker per day
portfolio_values = pd.DataFrame()
for ticker in tickers:
    portfolio_values[ticker] = data[ticker] * shares[ticker]

# Step 2: total portfolio value per day
total_value = portfolio_values.sum(axis=1)

# Step 3: daily weights = (ticker value / total value)
weights = portfolio_values.div(total_value, axis=0)

# Step 4: weighted portfolio daily return
portfolio_daily_return = (daily_returns * weights).sum(axis=1)

# 5. Align with benchmark return for analysis
combined_df = pd.concat([portfolio_daily_return, benchmark_returns], axis=1).dropna()
combined_df.columns = ['Portfolio', 'SP500']

# 6. Calculate Overall Portfolio Performance Metrics
trading_days = 45
risk_free_rate = 0.02

avg_daily_return = combined_df['Portfolio'].mean()
daily_volatility = combined_df['Portfolio'].std()
annualized_return = avg_daily_return * trading_days
annualized_volatility = daily_volatility * np.sqrt(trading_days)
sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility

# Information Ratio
excess_returns = combined_df['Portfolio'] - combined_df['SP500']
mean_excess_return = excess_returns.mean() * trading_days
tracking_error = excess_returns.std() * np.sqrt(trading_days)
information_ratio = mean_excess_return / tracking_error if tracking_error != 0 else np.nan

# Treynor Ratio (based on beta with S&P 500)
cov_matrix = np.cov(combined_df['Portfolio'], combined_df['SP500'])
beta = cov_matrix[0, 1] / np.var(combined_df['SP500']) if np.var(combined_df['SP500']) != 0 else np.nan
treynor_ratio = (annualized_return - risk_free_rate) / beta if beta and beta != 0 else np.nan


[*********************100%***********************]  9 of 9 completed


[*********************100%***********************]  1 of 1 completed


Historical Prices (last 5 rows):
Ticker            EFA       GOVT        HYG        IJH         LQD  \
Date                                                                 
2025-03-28  82.459999  22.859186  78.418442  58.209999  108.063812   
2025-03-31  81.730003  22.919001  78.508003  58.349998  108.273003   
2025-04-01  81.849998  22.969999  78.690002  58.660000  108.660004   
2025-04-02  82.040001  22.950001  78.849998  59.599998  108.910004   
2025-04-03  80.320000  23.120001  77.870003  55.590000  108.900002   

Ticker             SPY         TIP        VNQ        VWO  
Date                                                      
2025-03-28  555.659973  109.943810  89.709999  45.310001  
2025-03-31  559.390015  110.380997  90.540001  45.259998  
2025-04-01  560.969971  110.449997  90.580002  45.410000  
2025-04-02  564.520020  110.279999  91.129997  45.450001  
2025-04-03  536.700012  110.970001  88.019997  44.630001  

Daily Returns (last 5 rows):
Ticker           EFA      GOVT  

# Portfolio Performance

In [47]:

# Store all metrics in a dictionary
portfolio_stats = {
    "Average Daily Return": avg_daily_return,
    "Average Daily Std Dev (Volatility)": daily_volatility,
    "Average Annualized Return": annualized_return,
    "Average Annualized Std Dev": annualized_volatility,
    "Sharpe Ratio": sharpe_ratio,
    "Information Ratio": information_ratio,
    "Treynor Ratio": treynor_ratio,
    "Beta (vs SP500)": beta
}
Portfolio = pd.DataFrame(portfolio_stats,index=["Portfolio Stats"])
Portfolio

,Average Daily Return,Average Daily Std Dev (Volatility),Average Annualized Return,Average Annualized Std Dev,Sharpe Ratio,Information Ratio,Treynor Ratio,Beta (vs SP500)
Portfolio Stats,-0.000641,0.006808,-0.028837,0.045671,-1.069305,1.907904,-0.094311,0.517825
